# Optimizing Online Sports Retail Revenue

Sports clothing and athleisure attire is a huge industry, worth approximately $193 billion in 2021 with a strong growth forecast over the next decade! 
For this project, we play the role of a data analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. We will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

The database provided to us, sports, contains five tables, with product_id being the primary key for all of them:

<h4 id="retail_info"><code>retail_info</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>description</code></td>
<td><code>varchar</code></td>
<td>Description of the product</td>
</tr>
</tbody>
</table>
<h4 id="retail_finance"><code>retail_finance</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>listing_price</code></td>
<td><code>float</code></td>
<td>Listing price for product</td>
</tr>
<tr>
<td><code>sale_price</code></td>
<td><code>float</code></td>
<td>Price of the product when on sale</td>
</tr>
<tr>
<td><code>discount</code></td>
<td><code>float</code></td>
<td>Discount, as a decimal, applied to the sale price</td>
</tr>
<tr>
<td><code>revenue</code></td>
<td><code>float</code></td>
<td>Amount of revenue generated by each product, in US dollars</td>
</tr>
</tbody>
</table>
<h4 id="retail_reviews"><code>retail_reviews</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_name</code></td>
<td><code>varchar</code></td>
<td>Name of the product</td>
</tr>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>rating</code></td>
<td><code>float</code></td>
<td>Product rating, scored from <code>1.0</code> to <code>5.0</code></td>
</tr>
<tr>
<td><code>reviews</code></td>
<td><code>float</code></td>
<td>Number of reviews for the product</td>
</tr>
</tbody>
</table>
<h4 id="retail_traffic"><code>retail_traffic</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>last_visited</code></td>
<td><code>timestamp</code></td>
<td>Date and time the product was last viewed on the website</td>
</tr>
</tbody>
</table>
<h4 id="retail_brands"><code>retail_brands</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>product_id</code></td>
<td><code>varchar</code></td>
<td>Unique ID for product</td>
</tr>
<tr>
<td><code>brand</code></td>
<td><code>varchar</code></td>
<td>Brand of the product</td>
</tr>
</tbody>
</table>

***1. Counting missing values***

  We will be dealing with missing data as well as numeric, string, and timestamp data types to draw insights about the products in the online store. Let's start by finding out how complete the data is.


In [ ]:
-- Count all columns as total_rows
-- Count the number of non-missing entries for description, listing_price, and last_visited
-- Join info, finance, and traffic

SELECT COUNT(*) AS "total_rows", COUNT(description) AS "count_description", COUNT(listing_price) AS "count_listing_price", COUNT(last_visited) AS "count_last_visited"
FROM retail_info
JOIN retail_finance ON retail_info.product_id= retail_finance.product_id
JOIN retail_traffic ON retail_info.product_id= retail_traffic.product_id;


***2. Nike vs Adidas pricing***

  We can see the database contains 3,179 products in total. Of the columns we previewed, only one last_visited ; is missing more than five percent of its values. Now let's turn our attention to pricing. 
  
  How do the price points of Nike and Adidas products differ? Answering this question can help us build a picture of the company's stock range and customer market. We will run a query to produce a distribution of the listing_price and the count for each price, grouped by brand.


In [ ]:
-- Select the brand, listing_price as an integer, and a count of all products in finance 
-- Join brands to finance on product_id
-- Aggregate results by brand and listing_price, and sort the results by listing_price in descending order
-- Filter for products with a listing_price more than zero

SELECT brand, ROUND(listing_price) AS 'listing_price', COUNT(*) AS 'count'
FROM retail_brands
INNER JOIN retail_finance ON retail_brands.product_id= retail_finance.product_id
GROUP BY brand, listing_price
HAVING listing_price > 0
ORDER BY listing_price DESC;


***3. Labeling price ranges***
  It turns out there are 77 unique prices for the products in our database, which makes the output of our last query quite difficult to analyze. 
  
  Let's build on our previous query by assigning labels to different price ranges, grouping by brand and label. We will also include the total revenue for each price range and brand.


In [ ]:
-- Select the brand, a count of all products in the finance table, and total revenue
-- Create four labels for products based on their price range, aliasing as price_category
-- Join brands to finance on product_id
-- Group results by brand and price_category, sort by total_revenue and filter out products missing a value for brand

SELECT brand, COUNT(*), SUM(revenue) as total_revenue,
CASE WHEN listing_price < 42 THEN 'Budget'
    WHEN listing_price >= 42 AND listing_price < 74 THEN 'Average'
    WHEN listing_price >= 74 AND listing_price < 129 THEN 'Expensive'
    ELSE 'Elite' END AS price_category
FROM retail_finance
INNER JOIN retail_brands
    ON retail_finance.product_id = retail_brands.product_id
GROUP BY brand, price_category
HAVING brand IS NOT NULL
ORDER BY total_revenue DESC;




***4. Average discount by brand***

Interestingly, grouping products by brand and price range allows us to see that Adidas items generate more total revenue regardless of price category! Specifically, ‘Elite’ Adidas products priced \$129 or more typically generate the highest revenue, so the company can potentially increase revenue by shifting their stock to have a larger proportion of these products!

Note we have been looking at listing_price so far. The listing_price may not be the price that the product is ultimately sold for. To understand revenue better, let's take a look at the discount,  which is the percent reduction in the listing_price when the product is actually sold. We would like to know whether there is a difference in the amount of discount offered between brands, as this could be influencing revenue.


In [ ]:
-- Select brand and average_discount as a percentage
-- Join brands to finance on product_id
-- Aggregate by brand
-- Filter for products without missing values for brand

SELECT brand, AVG(discount) *100 AS "average_discount" 
FROM retail_brands
JOIN retail_finance ON retail_brands.product_id= retail_finance.product_id
GROUP BY brand
HAVING brand IS NOT NULL;


***5. Correlation between revenue and reviews***

Strangely, no discount is offered on Nike products! In comparison, not only do Adidas products generate the most revenue, but these products are also heavily discounted!

To improve revenue further, the company could try to reduce the amount of discount offered on Adidas products, and monitor sales volume to see if it remains stable. Alternatively, it could try offering a small discount on Nike products. This would reduce average revenue for these products, but may increase revenue overall if there is an increase in the volume of Nike products sold.

Now explore whether relationships exist between the columns in our database. We will check the strength and direction of a correlation between revenue and reviews. 


In [ ]:
-- Calculate the correlation between reviews and revenue as corr
-- Join the reviews and finance tables on product_id

SELECT  @firstValue := AVG(reviews),
        @secondValue := AVG(revenue),
        @division := (stddev_samp(reviews) * stddev_samp(revenue))
FROM retail_reviews
JOIN retail_finance ON retail_reviews.product_id= retail_finance.product_id;

SELECT SUM((reviews-@firstValue)*(revenue-@secondValue))/((COUNT(reviews)-1)*@division) AS review_revenue_corr
FROM retail_reviews
JOIN retail_finance ON retail_reviews.product_id= retail_finance.product_id;

***6. Ratings and reviews by product description length***

  Interestingly, there is a strong positive correlation between revenue and reviews. This means, potentially, if we can get more reviews on the company's website, it may increase sales of those items with a larger number of reviews.
  
  Perhaps the length of a product's description might influence a product's rating and reviews - if so, the company can produce content guidelines for listing products on their website and test if this influences revenue. Let's check this out!


In [ ]:
-- Calculate description_length
-- Convert rating to an integer and calculate average_rating
-- Join info to reviews on product_id and group the results by description_length
-- Filter for products without missing values for description, and sort results by description_length

SELECT  TRUNCATE(LENGTH(description), -2) AS description_length,
       ROUND(AVG(rating),2) AS average_rating
FROM retail_info
JOIN retail_reviews ON retail_info.product_id = retail_reviews.product_id
GROUP BY description_length
HAVING description_length IS NOT NULL
ORDER BY description_length; 


***7. Reviews by month and brand***

Unfortunately, there doesn't appear to be a clear pattern between the length of a product's description and its rating.

As we know a correlation exists between reviews and revenue, one approach the company could take is to run experiments with different sales processes encouraging more reviews from customers about their purchases, such as by offering a small discount on future purchases.

Let's take a look at the volume of reviews by month to see if there are any trends or gaps we can look to exploit.


In [ ]:
-- Select brand, month from last_visited, and a count of all products in reviews aliased as num_reviews
-- Join traffic with reviews and brands on product_id
-- Group by brand and month, filtering out missing values for brand and month
-- Order the results by brand and month

SELECT brand, MONTH(last_visited) AS 'month', COUNT(retail_reviews.product_id) AS 'num_reviews'
FROM retail_reviews
JOIN retail_brands ON retail_reviews.product_id= retail_brands.product_id
JOIN retail_traffic ON retail_reviews.product_id= retail_traffic.product_id
GROUP BY brand, month
HAVING brand IS NOT NULL
    AND month IS NOT NULL
ORDER BY brand, month;


***8. Footwear product performance***

Looks like product reviews are highest in the first quarter of the calendar year, so there is scope to run experiments aiming to increase the volume of reviews in the other nine months!

So far, we have been primarily analyzing Adidas vs Nike products. Now, let's switch our attention to the type of products being sold. As there are no labels for product type, we will create a Common Table Expression (CTE) that filters description for keywords, then use the results to find out how much of the company's stock consists of footwear products and the median revenue generated by these items.


In [ ]:
-- Create the footwear CTE, containing description and revenue
-- Filter footwear for products with a description containing %shoe%, %trainer, or %foot%
-- Also filter for products that are not missing values for description
-- Calculate the number of products and median revenue for footwear products

WITH median_calculation AS
    ( 
        SELECT 
            revenue, 
            @rownum:=@rownum+1 AS x,
            @total_rows:=@rownum
        FROM (
                SELECT 
                    revenue,
                    description
                FROM retail_info
                JOIN retail_finance ON retail_finance.product_id = retail_info.product_id
                WHERE description IS NOT NULL
                    AND revenue IS NOT NULL
                    AND description LIKE '%shoe%'
                    OR description LIKE '%trainer%'
                    OR description LIKE '%foot%'
                ORDER BY revenue
        ) footwear_selection
        , (SELECT @rownum:=0) tab
        WHERE revenue is NOT NULL
        ORDER BY revenue
    )
SELECT AVG(revenue) AS median_footwear, @total_rows AS number_footwear
FROM median_calculation
WHERE x IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) )

***9. Clothing product performance***

Recall from the first task that we found there are 3,117 products without missing values for description. Of those, 2,700 are footwear products, which accounts for around 85% of the company's stock. They also generate a median revenue of over $3000 dollars!

This is interesting, but we have no point of reference for whether footwear's median_revenue is good or bad compared to other products. So, for our final task, let's examine how this differs to clothing products. We will re-use footwear, adding a filter afterward to count the number of products and median_revenue of products that are not in footwear.


In [ ]:
-- Copy the footwear CTE from the previous task
-- Calculate the number of products in info and median revenue from finance
-- Inner join info with finance on product_id
-- Filter the selection for products with a description not in footwear

WITH median_calculation AS
    ( 
        SELECT 
            revenue, 
            @rownum:=@rownum+1 AS x,
            @total_rows:=@rownum
        FROM (
                WITH footwear AS
                    (
                        SELECT 
                            revenue,
                            description
                        FROM retail_info
                        JOIN retail_finance ON retail_finance.product_id = retail_info.product_id
                        WHERE description IS NOT NULL
                            AND revenue IS NOT NULL
                            AND description LIKE '%shoe%'
                            OR description LIKE '%trainer%'
                            OR description LIKE '%foot%'
                        ORDER BY revenue
                    )
                SELECT revenue, description
                FROM retail_info
                JOIN retail_finance ON retail_finance.product_id = retail_info.product_id
                WHERE description NOT IN (SELECT description FROM footwear)
             ) footwear_selection
        ,(SELECT @rownum:=0) tab
        WHERE revenue is NOT NULL
        ORDER BY revenue
    )
SELECT AVG(revenue) AS median_clothing, @total_rows AS number_clothing
FROM median_calculation
WHERE x IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) )
